<a href="https://colab.research.google.com/github/astrissha/Happy-Sad-Detection/blob/main/happysaddetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow scikit-learn opencv-python

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

# Define your data directories
data_dir = "/content/drive/MyDrive/dataset" # replace with your directory
happy_dir = os.path.join(data_dir, "happy")
sad_dir = os.path.join(data_dir, "sad")

# Image dimensions
img_width, img_height = 150, 150

# Load images and labels
def load_images_and_labels(directory, label):
    images = []
    labels = []
    for filename in os.listdir(directory):
        img_path = os.path.join(directory, filename)
        img = cv2.imread(img_path)
        if img is not None: #check for corrupted images.
            img = cv2.resize(img, (img_width, img_height))
            img = img / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(label)
        else:
            print(f"could not load image: {img_path}")
    return images, labels

happy_images, happy_labels = load_images_and_labels(happy_dir, 1) # 1 for happy
sad_images, sad_labels = load_images_and_labels(sad_dir, 0)     # 0 for sad

# Combine and convert to NumPy arrays
images = np.array(happy_images + sad_images)
labels = np.array(happy_labels + sad_labels)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Data augmentation (optional)
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

datagen.fit(X_train)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5), # Regularization
    Dense(1, activation='sigmoid') # Binary classification (happy/sad)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
batch_size = 32
epochs = 30 # Adjust as needed

model.fit(datagen.flow(X_train, y_train, batch_size=batch_size),
          epochs=epochs,
          validation_data=(X_test, y_test))

#or without data augmentation:

#model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test,y_test), batch_size=batch_size)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 8s 8s/step - accuracy: 0.4839 - loss: 0.6959 - val_accuracy: 0.3750 - val_loss: 1.3323
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5161 - loss: 1.1154 - val_accuracy: 0.6250 - val_loss: 0.7533
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5161 - loss: 0.9429 - val_accuracy: 0.6250 - val_loss: 0.6801
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.5484 - loss: 0.6977 - val_accuracy: 0.3750 - val_loss: 0.7284
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step - accuracy: 0.4839 - loss: 0.6688 - val_accuracy: 0.3750 - val_loss: 0.8024
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5161 - loss: 0.6712 - val_accuracy: 0.3750 - val_loss: 0.7674
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.5161 - loss: 0.6590 - val_accuracy: 0.3750 - val_loss: 0.7127
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.5484 - loss: 0.6710 - val_accuracy: 0.3750 - val_loss: 0.6752
Epoch 9/30
1/1 ━

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step - accuracy: 0.7500 - loss: 1.7258
Test Loss: 1.7258
Test Accuracy: 0.7500


In [ ]:
model.save("happy_sad_model.h5")

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import os

# Load the trained model
model = tf.keras.models.load_model("happy_sad_model.h5")  # Replace with your model file

# Image dimensions (should match training dimensions)
img_width, img_height = 150, 150

# Function to preprocess and predict
def predict_emotion(image_path):
    try:
        img = cv2.imread(image_path)
        if img is None:
            print(f"Error: Could not load image from {image_path}")
            return None

        img = cv2.resize(img, (img_width, img_height))
        img = img / 255.0  # Normalize
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        prediction = model.predict(img)
        if prediction[0][0] > 0.5:
            return "Happy"
        else:
            return "Sad"
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Test the model with example images
test_image_happy = "test_happy.jpg"  # Replace with your happy image path
test_image_sad = "test_sad.jpg"    # Replace with your sad image path

if os.path.exists(test_image_happy):
    emotion_happy = predict_emotion(test_image_happy)
    if emotion_happy:
        print(f"{test_image_happy}: Predicted Emotion: {emotion_happy}")
else:
    print(f"Error: {test_image_happy} not found.")

if os.path.exists(test_image_sad):
    emotion_sad = predict_emotion(test_image_sad)
    if emotion_sad:
        print(f"{test_image_sad}: Predicted Emotion: {emotion_sad}")
else:
    print(f"Error: {test_image_sad} not found.")

# Test with a single image file provided by the user.
def test_single_image():
    image_path = input("Enter the path to the image you want to test: ")

    if os.path.exists(image_path):
        emotion = predict_emotion(image_path)
        if emotion:
            print(f"{image_path}: Predicted Emotion: {emotion}")
    else:
        print(f"Error: {image_path} not found.")

test_single_image()

Error: test_happy.jpg not found.
Error: test_sad.jpg not found.
Enter the path to the image you want to test: /content/drive/MyDrive/dataset/test_image.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 118ms/step
/content/drive/MyDrive/dataset/test_image.jpg: Predicted Emotion: Sad


In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import cv2
import numpy as np
from PIL import Image
import os

# Load the trained model (ensure it's uploaded to Colab)
model = tf.keras.models.load_model("happy_sad_model.h5")  # Replace if needed

# Image dimensions (match training dimensions)
img_width, img_height = 150, 150

# Function to preprocess and predict
def predict_emotion(image):
    try:
        img = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR) #convert PIL image to cv2 image.
        img = cv2.resize(img, (img_width, img_height))
        img = img / 255.0  # Normalize
        img = np.expand_dims(img, axis=0)  # Add batch dimension

        prediction = model.predict(img)
        if prediction[0][0] > 0.5:
            return "Happy"
        else:
            return "Sad"
    except Exception as e:
        st.error(f"An error occurred: {e}")
        return None

def main():
    st.title("Happy/Sad Emotion Detector")

    uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

    if uploaded_file is not None:
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image.", use_column_width=True)
        st.write("")
        st.write("Predicting...")

        emotion = predict_emotion(image)

        if emotion:
            st.write(f"Predicted Emotion: {emotion}")
        else:
            st.write("Prediction failed.")

if __name__ == '__main__':
    main()

#Run in colab:
# !streamlit run your_script_name.py --server.port 8501 --server.address 0.0.0.0

Overwriting app.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp app.py /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
